# Aufgabe-C-Sturm

## Allgemeines

Eine allgemeine Beschreibung der Laboraufgaben inklusive des Vorgehens, den Bewertungsrichtlinien und der Abgabe finden Sie  <a href="ML-allgemein.ipynb">hier</a>

## Datenquelle


* Laden Sie ihre Daten von http://141.72.190.207/ml_lab/C_sturm herunter
    * Die Daten sind geschützt. 
        * Sie müssen evtl. in einem Netzwerk der DHBW (z.B. WLAN, VPN, ...) angemeldet sein. 
        * Sie können sich auf der Webseite mit dem Benutzernamen dhbw und dem Zugangsnamen "ml_LaB_4$" anmelden. 
* Die Daten sind in einem anwendungsspezifischen Format gespeichert.
    * Sie finden evtl. Informationen über die Daten in einer "README" Datei. 
    * Finden Sie keine solche Datei sind die Daten selbst erklärend. 
    
    



## Aufgabe

Sagen Sie die Windgeschwindigkeit eines Sturms (in Knoten) aufgrund von Satellitenfotos vorher.
Der Datensatz besteht aus Fotos von 494 verschiedenen Stürmen im Atlantik und Pazifik mit ihren zugehörigen Windgeschwindigkeiten.
Jedes Bild hat `366 x 366` Pixel, und es sind 70.257 Trainingsdaten und 45.377 Testdaten vorhanden.
Die Bilder wurden zu mehreren Zeitpunkten während der Lebensdauer eines Sturms aufgenommen.

Für jeden Sturm im Trainings und Testdatensatz erhalten Sie eine Zeitreihe von Bildern mit der jeweiligen assoziierten relativen Zeit seit Beginn des Sturms.
Ihr Modell sollte neben den reinen Bilddaten also auch den zeitlichen Verlauf des Sturms betrachten, um die Vorhersage für künftige Vorhersagezeitpunkte zu erstellen

Die Bilder sind nach folgendem Schema benannt: `{image_id}.jpg`.
Diese IDs bestehen aus einer Sturm_ID und einer Bildnummer entsprechend der zeitlichen Bildreihenfolge.

Ihr Ziel ist es, für die Testdaten die korrekte Windgeschwindigkeiten vorherzusagen.

# Lösung

* Beginnen Sie hier mit Ihrer Dokumentation und Implementierung! 

Geplantes Vorgehen: 

- Daten vorbereitung
- CNN-Modell trainieren (für räumliche Merkmale der Bilder)
- LSTM-Modell trainieren (für zeitlichen Verlauf der Bilder)
- Kombinierung der Modelle (Fully Connected-Layer um CNN in LSTM einzuspeisen, Ausgabe von LSTM ist finale Vorhersage)
- Kombinierte Modell auf Trainingsdaten trainieren 

In [1]:
import json
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_data(image_path, label_path):
    # Lade das Bild und konvertiere es in ein Numpy-Array
    img = load_img(image_path, target_size=(366, 366))
    img = img_to_array(img)

    # Lade die Labels aus der JSON-Datei
    with open(label_path) as f:
        labels = json.load(f)

    # Extrahiere die Windgeschwindigkeit aus den Labels
    wind_speed = labels["wind_speed"]

    return img, wind_speed

In [4]:
import os

# Erstelle leere Arrays, um die Daten zu speichern
train_images = []
train_labels = []
test_images = []
test_labels = []

# Durchlaufe die Trainingsdaten
train_data_dir = "storm_train_data"
train_label_dir = "storm_train_labels"
i = 0
for storm_folder in os.listdir(train_data_dir):
    for img_file in os.listdir(os.path.join(train_data_dir, storm_folder)):
        img_path = os.path.join(train_data_dir, storm_folder, img_file)
        label_path = os.path.join(train_label_dir, storm_folder, "labels.json")
        img, wind_speed = load_data(img_path, label_path)
        train_images[i] = img
        train_labels[i] = wind_speed
        i += 1

# Durchlaufe die Testdaten
test_data_dir = "storm_test_data"
test_label_dir = "storm_test_labels"
i = 0
for storm_folder in os.listdir(test_data_dir):
    for img_file in os.listdir(os.path.join(test_data_dir, storm_folder)):
        img_path = os.path.join(test_data_dir, storm_folder, img_file)
        label_path = os.path.join(test_label_dir, storm_folder, "labels.json")
        img, wind_speed = load_data(img_path, label_path)
        test_images[i] = img
        test_labels[i] = wind_speed
        i += 1

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001C948433B00>